In [1]:
install.packages("jsonlite")
install.packages("dplyr")

require(jsonlite)
require(dplyr)

# import commits from JSON
comments <- fromJSON("http://pyro.primeprocessor.com:8888/files/comments.json")

# create corpus
comments <- VCorpus(VectorSource(comments$comment))

# remove puncutation, stopwords, numbers, whitespace
mod <- tm_map(comments, tolower)
mod <- tm_map(mod, removePunctuation)
mod <- tm_map(mod, removeNumbers)
mod <- tm_map(res, removeWords, stopwords("english"))
mod <- tm_map(mod, stripWhitespace)

mod <- tm_map(mod, PlainTextDocument)
dtm <- DocumentTermMatrix(mod)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning message:
In install.packages("jsonlite"): 'lib = "/usr/local/lib/R/site-library"' is not writable

ERROR: Error in install.packages("jsonlite"): unable to install packages


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning message:
In install.packages("dplyr"): 'lib = "/usr/local/lib/R/site-library"' is not writable

ERROR: Error in install.packages("dplyr"): unable to install packages


Loading required package: jsonlite

Attaching package: ‘jsonlite’

The following object is masked from ‘package:utils’:

    View

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



...

ERROR: Error in eval(expr, envir, enclos): could not find function "tm_map"


ERROR: Error in eval(expr, envir, enclos): could not find function "tm_map"


ERROR: Error in eval(expr, envir, enclos): could not find function "tm_map"


ERROR: Error in eval(expr, envir, enclos): could not find function "DocumentTermMatrix"


ERROR: Error in eval(expr, envir, enclos): could not find function "weightTfIdf"


The term-frequency-inverse-document-frequency matrix assigns weights to words in a document according to their frequency of appearance.  To account for the fact that some words appear more frequently in a given corpus (a collection of documents), each term frequency is multiplied by the inverse document frequency.  This tampers the weights.  

In [ ]:
dtm_tfxidf <- weightTfIdf(dtm)

In [ ]:
term_wt <- apply(dtm_tfxidf,2,sum)
length(term_wt[which(term_wt>2.5)])

dtm_tfxidf <- dtm_tfxidf[,term_wt>2.5]

tss <- c()
for (i in 1:50) {
  fit <- kmeans(dtm_tfxidf, i)
  tss[i] <- sum(fit$tot.withinss)
}

# no elbow in chart, pick k = 10
plot(tss, main="Optimal K's", ylab="Total SS Within", xlab="# of ks")

fit <- kmeans(dtm_tfxidf, 10)
clusters <- fit$cluster

clust.df <- as.matrix(dtm_tfxidf)
clust.df<- as.data.frame(clust.df)
all <- cbind(fit$cluster,clust.df)

names(all)[1] <- "cluster"

freq_terms <- list()
for (i in 1:10) {
  selection <- all %>% filter(cluster==i)
  vector <- colSums(selection[,-1]) %>% sort(., decreasing=T)
  freq_terms[[i]] <- vector[1:5]
}

